# Агрегирование в pandas
Pandas поддерживает все возможности по агрегированию, которые есть в NumPy. Кроме простых агрегирующих функций в pandas также есть возможность группировки и трансформации данных, которая поволяет выполнять очень сложные вычисления над данными.

Создадим небольшой набор данных для простых примеров с агрегирующими функциями

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)

In [2]:
countries = ['Afghanistan', 'Kazakhstan', 'Kyrgyzstan', 'Tajikistan', 'Turkmenistan', 'Uzbekistan']
area = [652864, 2724900, 199951, 143100, 491210, 448978]
population = [34656032, 17987736, 6019480, 8734951, 5662544, 32979000]
gdp = [21, 156.189, 7.061, 27.802, 42.355, 68.324]
gini = [29, 26.4, 27.4, 30.8, 40.8, 36.7]
central_asia = pd.DataFrame(data={'area': area, 
                                  'population': population,
                                  'gdp': gdp,
                                  'gini': gini}, 
                            index=countries)
central_asia

,area,gdp,gini,population
Afghanistan,652864,21.000,29.0,34656032
Kazakhstan,2724900,156.189,26.4,17987736
Kyrgyzstan,199951,7.061,27.4,6019480
Tajikistan,143100,27.802,30.8,8734951
Turkmenistan,491210,42.355,40.8,5662544
Uzbekistan,448978,68.324,36.7,32979000


Простые агрегирующие функции такие как `min`, `max`, `mean`, `sum` и т.д. можно вызывать напрямую как методы `Series` или `DataFrame ` объектов

In [3]:
print('min population:', central_asia['population'].min())
print('max population:', central_asia['population'].max())
print('mean population:', central_asia['population'].mean())
print('total population:', central_asia['population'].sum())

min population: 5662544
max population: 34656032
mean population: 17673290.5
total population: 106039743


Если вызвать агрегирующую функцию у объекта `DataFrame`, то соответствующая функция будет вычислена по всем колонкам `DataFrame`

In [4]:
central_asia.sum()

area          4.661003e+06
gdp           3.227310e+02
gini          1.911000e+02
population    1.060397e+08
dtype: float64

В pandas есть метод `describe`, который вычисляет основные агрегирующие функции по всем данным

In [5]:
central_asia.describe()

,area,gdp,gini,population
count,6.000000e+00,6.00000,6.000000,6.000000e+00
mean,7.768338e+05,53.78850,31.850000,1.767329e+07
std,9.730173e+05,54.33546,5.698333,1.328714e+07
min,1.431000e+05,7.06100,26.400000,5.662544e+06
25%,2.622078e+05,22.70050,27.800000,6.698348e+06
50%,4.700940e+05,35.07850,29.900000,1.336134e+07
75%,6.124505e+05,61.83175,35.225000,2.923118e+07
max,2.724900e+06,156.18900,40.800000,3.465603e+07


# Group by - группировка данных в pandas
До сих пор мы видели агрегирующие функции, которые работают со всем набором данных. С помощью group by мы можем вычислять  функции над подмножеством набора данных. Процесс группировки состоит из трех шагов:
1. Разбиение данных на группы на основе заданной логики (Splitting)
2. Применение функции к каждой группе (Applying)
3. Комбинирование результата применененных функций в одну общую структуру (Combining)

## [Группировка](https://pandas.pydata.org/pandas-docs/stable/groupby.html)
Для работы с группировками создадим набор данных содержищий информацию о процессе создания ПО. Мы уже видели этот набор при изучении иерархических индексов. Здесь мы не будем создавать иерархический индекс, вместо этого будем напрямую работать с данными в простом табличном виде.

In [6]:
from itertools import product
teams = ['Backend', 'Frontend', 'Mobile']
task_types = ['Bug', 'Improvement', 'Feature']
stages = ['Backlog', 'Development', 'QA', 'Release']

process_board_df = pd.DataFrame(data=list(product(teams, task_types, stages)), 
                                columns=['team', 'task_type', 'stage'])
process_board_df['task_count'] = np.random.randint(0, 5, size=36)
process_board_df

,team,task_type,stage,task_count
0,Backend,Bug,Backlog,4
1,Backend,Bug,Development,0
2,Backend,Bug,QA,3
3,Backend,Bug,Release,3
4,Backend,Improvement,Backlog,3
5,Backend,Improvement,Development,1
6,Backend,Improvement,QA,3
7,Backend,Improvement,Release,2
8,Backend,Feature,Backlog,4
9,Backend,Feature,Development,0


Для создания группировки используется метод `groupby`, куда нужно передать как минимум название одной колонки для группировки

In [7]:
process_board_df.groupby('team')

В результате мы получаем объект `DataFrameGroupBy`, к которому можно применять различные функции. Чаще всего группировка и применение функции выполняется как цепочка вызовов. Например, мы можем посчитать общее количество задач в каждой из комманд

In [8]:
process_board_df.groupby('team').sum()

,task_count
team,
Backend,27
Frontend,16
Mobile,18


Мы можем передать более, чем одну колонку для группировки. Например, можно вычислить количество текущих задач на каждом этопе в каждой из комманд следующим образом

In [9]:
process_board_df.groupby(by=['team', 'stage']).sum()

task_count
team     stage                  
Backend  Backlog              11
         Development           1
         QA                    6
         Release               9
Frontend Backlog               6
         Development           1
         QA                    4
         Release               5
Mobile   Backlog               6
         Development           7
         QA                    4
         Release               1

Group by - очень гибкая конструкция. Мы можем группировать не только по названию колонки, но и с помощью другой последовательности такого же размера. При этом элементы последовательности с одинаковым значением попадут в одну группу. 

Рассмотрим следующий пример. Часто, команды, которые используют гибкие методологии внедрения (например, Kanban) определяют для себя лимиты по максимальному количеству текущих задач, которые можно выполнять (WIP limit, work in progress limit). Это делается для того, чтобы быстрее завершать начатые задачи, не тратя времени и сил на постоянное переключение фокуса из одной задачи в другую. Допустим, что в рассматриваемой нами команде этот лимит на всех этапах равен трем. В таком случае мы можем определить количество этапов, где этот порог превышен следующим образом. Сначала создадим булевую маску

In [10]:
wip_limit_exceeded = process_board_df['task_count'] > 3
wip_limit_exceeded

0      True
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8      True
9     False
10    False
11     True
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19     True
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
Name: task_count, dtype: bool

Затем используем эту маску в качестве ключа группировки. При этом этапы, где порог превышен и не превышен, попадут в разные группы. Такое возможно благодаря выравниванию индексов

In [11]:
process_board_df.groupby(wip_limit_exceeded).count()['task_count']

task_count
False    32
True      4
Name: task_count, dtype: int64

Мы можем комбинировать ключи и получить количество задач с превышенным порогом в разрезе каждой комманды

In [12]:
process_board_df.groupby(by=['team', wip_limit_exceeded]).count()['task_count']

team      task_count
Backend   False          9
          True           3
Frontend  False         11
          True           1
Mobile    False         12
Name: task_count, dtype: int64

В качестве ключа для группировки можно использовать функцию. Эта функция будет получать индекс каждой строки и должна вернуть значение, которое будет использовано, чтобы отнести текущую строку к данной группе. Например, мы можем определить превышение порога задач для всех этапов кроме Backlog, так как в Backlog добавляются задачи, которые будут выполнены в будущем

In [13]:
def wip_limit(idx):
    row = process_board_df.iloc[idx]
    return (row['stage'] != 'Backlog') and (row['task_count'] > 3)

process_board_df.groupby(wip_limit).count()['task_count']

False    34
True      2
Name: task_count, dtype: int64

С помощью атрибута `groups` можно получить доступ к каждой из групп

In [14]:
gr = process_board_df.groupby(by='stage')
gr.groups

{'Backlog': Int64Index([0, 4, 8, 12, 16, 20, 24, 28, 32], dtype='int64'),
 'Development': Int64Index([1, 5, 9, 13, 17, 21, 25, 29, 33], dtype='int64'),
 'QA': Int64Index([2, 6, 10, 14, 18, 22, 26, 30, 34], dtype='int64'),
 'Release': Int64Index([3, 7, 11, 15, 19, 23, 27, 31, 35], dtype='int64')}

С помощью метода `get_group` можно получить отдельную группу по названию группы

In [15]:
gr.get_group('QA')

,team,task_type,stage,task_count
2,Backend,Bug,QA,3
6,Backend,Improvement,QA,3
10,Backend,Feature,QA,0
14,Frontend,Bug,QA,0
18,Frontend,Improvement,QA,1
22,Frontend,Feature,QA,3
26,Mobile,Bug,QA,0
30,Mobile,Improvement,QA,3
34,Mobile,Feature,QA,1


Группировку можно использовать для итерации по группам. Таким образом можно обработать каждую группу по отдельности. Это может быть полезно, например, для разбиения очень большого набора данных на группы и обработки каждой группы параллельно в отдельном процессе

In [16]:
for name, group in gr:
    print(name, group.shape)

Backlog (9, 4)
Development (9, 4)
QA (9, 4)
Release (9, 4)


Группировка очень естественно сочетается с иерархическим индексом. Можно передать номер уровня в иерархическом индексе для группировки. Например, ниже в качестве уровней иерархического индекса используются название команд и типы задач. Мы можем сгруппировать эти данные по второму уровню, который соответствует типу задачи

In [17]:
multi_index_df = process_board_df.set_index(['team', 'task_type'])
multi_index_df.groupby(level=1).mean()

,task_count
task_type,
Bug,1.666667
Feature,1.416667
Improvement,2.000000


Если у уровня индекса есть название, то можно использовать при группировке

In [18]:
multi_index_df.index.names = ['team', 'task_type']
multi_index_df.groupby(level='task_type').mean()

,task_count
task_type,
Bug,1.666667
Feature,1.416667
Improvement,2.000000


## [Агрегирование по группам](https://pandas.pydata.org/pandas-docs/stable/groupby.html#aggregation)
До сих пор мы рассматривали простые встроенные функции для агрегации по группам. Однако у группировки есть общий метод `agg`, который принимает различные аргументы и с помощью них позволяет выполнять значительно более широкий набор действий.

В `agg` можно передать свою функцию для агрегации. Например, мы можем использовать метод `np.sum`, чтобы суммировать данные

In [19]:
process_board_df.groupby(by=['team', 'stage']).agg(np.sum)

task_count
team     stage                  
Backend  Backlog              11
         Development           1
         QA                    6
         Release               9
Frontend Backlog               6
         Development           1
         QA                    4
         Release               5
Mobile   Backlog               6
         Development           7
         QA                    4
         Release               1

Можно использовать  `lambda` выражение в качестве функции агрегирования. Например, следующий пример возвращает разницу между средним количеством задач в каждой команде со средним количеством задач во всех группах

In [20]:
mean_task_count = process_board_df['task_count'].mean()
print('mean task_count: ', mean_task_count)
process_board_df.groupby('team').agg(lambda x: x.mean() - mean_task_count)

mean task_count:  1.6944444444444444


,task_count
team,
Backend,0.555556
Frontend,-0.361111
Mobile,-0.194444


### Агрегирование по нескольким функциям
Метод `agg` позволяет агрегировать сразу по нескольким колонкам. Для этого достаточно передать список список агрегирующих функций

In [21]:
process_board_df.groupby('team').agg(['sum', 'mean', 'max'])

task_count              
                sum      mean max
team                             
Backend          27  2.250000   4
Frontend         16  1.333333   4
Mobile           18  1.500000   3

Можно комбинировать названия стандартных функций с другими функциями в списке для агрегирования

In [22]:
process_board_df.groupby('team').agg([np.sum, 'mean', lambda x: x.mean() - mean_task_count])

task_count                    
                sum      mean  <lambda>
team                                   
Backend          27  2.250000  0.555556
Frontend         16  1.333333 -0.361111
Mobile           18  1.500000 -0.194444

Для разных колонок можно вычислить разный набор агрегирующих функций. Для этого в метод `agg` необходимо передать `dict` с названием колонки в качестве ключа и со списком функций, которые нудно применить к этой колонке. Для демонстрации этой возможности добавим в наш `DataFrame` дополнительную колонку со средним временем выполнения задачи на каждом этапе

In [23]:
process_board_df['task_mean_time'] = 2 * np.random.random(process_board_df.shape[0]) + 3
process_board_df.head()

,team,task_type,stage,task_count,task_mean_time
0,Backend,Bug,Backlog,4,4.043697
1,Backend,Bug,Development,0,3.829324
2,Backend,Bug,QA,3,3.529111
3,Backend,Bug,Release,3,4.548467
4,Backend,Improvement,Backlog,3,3.912301


Теперь можно вычислить общее и среднее количество задач, а также макисмальное и минимальное среднее время выполнения задач в каждой команде следующим образом

In [24]:
process_board_df.groupby('team').agg({'task_count': ['sum', 'mean'], 'task_mean_time': [np.max, 'min']})

task_count           task_mean_time          
                sum      mean           amax       min
team                                                  
Backend          27  2.250000       4.887496  3.037580
Frontend         16  1.333333       4.395262  3.120451
Mobile           18  1.500000       4.976748  3.204090

Обратите внимание, что названия функций и сами функции агрегации можно смешивать в списке. Pandas при этом использует имя этой функции в качестве имени колонки. Можно задать свое название для колонки агрегации. Для этого в списке нужно передать кортежи с названием колонки в первом значении и с самой функцией во втором

In [25]:
process_board_df.groupby('team').agg({'task_count': [('total_task_count', 'sum'), ('mean_task_count', 'mean')], 
                                      'task_mean_time': [('max_mean_time', np.max), ('min_mean_time', 'min')]})

task_count                 task_mean_time              
         total_task_count mean_task_count  max_mean_time min_mean_time
team                                                                  
Backend                27        2.250000       4.887496      3.037580
Frontend               16        1.333333       4.395262      3.120451
Mobile                 18        1.500000       4.976748      3.204090

## [Трансформация](https://pandas.pydata.org/pandas-docs/stable/groupby.html#transformation)
Метод `transform` возвращает агрегированные результаты, которые имеют такой же размер и такие же индексы, что и оригинальный набор данных. Это очень полезно, когда нужно посчитать агрегирующую функцию по колонке и присвоить результат как новую колонку в оригинальный `DataFrame`

In [26]:
process_board_df['total_task_count'] = process_board_df.groupby('team')['task_count'].transform('sum')
process_board_df.head()

,team,task_type,stage,task_count,task_mean_time,total_task_count
0,Backend,Bug,Backlog,4,4.043697,27
1,Backend,Bug,Development,0,3.829324,27
2,Backend,Bug,QA,3,3.529111,27
3,Backend,Bug,Release,3,4.548467,27
4,Backend,Improvement,Backlog,3,3.912301,27


Часто перед тем, как передать данные в алгоритмы машинного обучения требуется отмасштабировать данные, чтобы значения всех данных попадало в один диапазон, например, в диапазон [0, 1] или [-1, 1]. Один из способов масштабирования данных называется стандартизацией, когда из оригинального значения вычитается среднее и затем делится на стандартное отклонение: 
$$x'=\frac{x-\bar{x}}{\sigma}$$
Полученные данные будут иметь среднее значение 0 и стандартное отклонение 1. Мы можем создать колонку со стандартизированным значением с помощью метода `transform` следующим образом

In [27]:
process_board_df['task_count_standard'] = process_board_df.groupby('team')['task_count'] \
                                                          .transform(lambda x: (x - x.mean()) / x.std())
process_board_df.head()

,team,task_type,stage,task_count,task_mean_time,total_task_count,task_count_standard
0,Backend,Bug,Backlog,4,4.043697,27,1.092006
1,Backend,Bug,Development,0,3.829324,27,-1.404008
2,Backend,Bug,QA,3,3.529111,27,0.468003
3,Backend,Bug,Release,3,4.548467,27,0.468003
4,Backend,Improvement,Backlog,3,3.912301,27,0.468003


Когда мы рассматривали работу с пропусками было показано, что один из методов заполнения пропущенных данных это замена их со средним значением. При этом среднее значение было вычислено по всему набору данных. Вместо этого, более корректно может быть использование среднего значения по группе. Например, рассмотрим следующий набор данных с пропусками

In [28]:
df = pd.DataFrame(np.arange(10).reshape((5, 2)))
df.iloc[0, 0] = np.nan
df.iloc[3, 1] = np.nan
df

,0,1
0,NaN,1.0
1,2.0,3.0
2,4.0,5.0
3,6.0,NaN
4,8.0,9.0


Мы сгруппируем данные по четным и нечетным индексам и заменим пропуски средними значениями в этих группах

In [29]:
df.groupby(by=lambda i: i % 2).transform(lambda x: x.fillna(x.mean()))

,0,1
0,6.0,1.0
1,2.0,3.0
2,4.0,5.0
3,6.0,3.0
4,8.0,9.0
